In [ ]:
!pip install jschon

In [2]:
from jschon import create_catalog,JSON, JSONSchema
import json,os
import pathlib
import inspect



In [3]:
create_catalog('2020-12')


Catalog('catalog')

In [130]:
keywords_to_avoid = ['not','anyOf','oneOf','allOf','patternPropreties']
def extract_errors_map(error:dict, error_map:dict):
    # print(error)
    # if there's errors in subschema treat them since they are the origin of the error  
    if 'errors' in error:
        for e in error['errors']:
            extract_errors_map(e, error_map)
    else : 
        keywordLocation = str(error['keywordLocation'])
        key = keywordLocation.split('/')[-1]
        
        if error["valid"] == True :
            if keywordLocation.__contains__("not"):
                if error_map.get(key) is None : 
                    error_map[key]=[error['error'] if 'error' in error else '']
                else : 
                    error_map[key].append(error['error'] if 'error' in error else '') 
                
            pass 
        else :      
            if key and  'error' in error and key not in keywords_to_avoid:
                if error_map.get(key) is None : 
                    error_map[key]=[error['error'] if 'error' in error else '']
                else : 
                    error_map[key].append(error['error'] if 'error' in error else '')
    
    # print("there's still error")
   

def extract_errors(errors): 
    error_map= {}
    for error in errors : 
        extract_errors_map(error,error_map)
    return list(error_map.keys())

In [108]:
def validate_instance(schema_dir:str,instance_dir): 
    with open(schema_dir,'r',encoding='utf-8') as f : 
        schema= json.load(f)
    schema['$schema'] ="https://json-schema.org/draft/2020-12/schema"
    schema['$id'] ="https://jschon.dev/demo"
    demo_schema = JSONSchema(schema)
    # print(demo_schema)
    with open (instance_dir,'r',encoding='utf-8') as f : 
        instance=json.load(f)
        
    result = demo_schema.evaluate(
        JSON(instance)
    )
    return result 
def get_errors_validation(schema_dir:str,instance_dir:str): 
    result = validate_instance(schema_dir,instance_dir)
    # print(result.output('detailed'))# if errors in result.output('detailed')
    if(result.output("verbose")['valid']==False or "errors" in result.output("verbose")): 
        errors = extract_errors(result.output('verbose')['errors'])
    else : 
        errors=[]
    return errors

In [129]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))
schema_path = src_file_path+'/dataset/7/schema.json'
instance_path = src_file_path+'/dataset/7/instance.json'
output = validate_instance(schema_path,instance_path).output('verbose')
with open(src_file_path+'/dataset/7/errors.json','w') as f: 
    json.dump(output,f)

In [131]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))

# print(src_file_path)
dataset_dir =  src_file_path+'/dataset'
for dir in os.listdir(dataset_dir) : 
    if dir=='instances' or dir=='schemas' : 
        continue 
    print(f"Treating schema : {dir}.json")
    schema_dir = f'{dataset_dir}/{dir}/schema.json'
    instance_dir=f'{dataset_dir}/{dir}/instance.json'
    errors=get_errors_validation(schema_dir,instance_dir)
    print(errors)

Treating schema : 1.json
['required', 'type']
Treating schema : 2.json
['type', 'minProperties']
Treating schema : 3.json
['type', 'required']
Treating schema : 4.json
[]
Treating schema : 5.json
['required']
Treating schema : 6.json
['type']
Treating schema : 7.json
['type']
